In [2]:
#importing the libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd 
import traceback
# import PyPDF2
# from dotenv import load_dotenv

In [3]:
from langchain.chat_models import ChatOpenAI

In [5]:
keys=os.getenv("OPENAI_KEY_1")


In [6]:
llm=ChatOpenAI(openai_api_key=keys, model_name="gpt-3.5-turbo",temperature=0.3)

d:\work\projects\project_ChatBot\ChatBot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# designing input and output prompt

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [11]:
#below prompt is for quiz evluation

TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [13]:
review_chain=LLMChain(llm=llm , prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [15]:
filePath=r"D:\work\projects\project_ChatBot\data.txt"

In [16]:
with open(filePath, 'r') as file:
    TEXT=file.read()

In [17]:
#converting the python dictionary nto json formate
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [18]:
NUMBER=5
SUBJECT="spcae craft"
TONE="simple"

In [19]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain, if you want to trak your token usage
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

d:\work\projects\project_ChatBot\ChatBot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [20]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2570
Prompt Tokens:1796
Completion Tokens:774
Total Cost:0.004242


In [21]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[13] Other researchers who have studied human cognitive system

In [22]:
quiz=response.get("quiz")

In [23]:
quiz

'{\n    "1": {\n        "mcq": "Who coined the term machine learning in 1959?",\n        "options": {\n            "a": "Donald Hebb",\n            "b": "Tom M. Mitchell",\n            "c": "Arthur Samuel",\n            "d": "Walter Pitts"\n        },\n        "correct": "c"\n    },\n    "2": {\n        "mcq": "What was the name of the experimental \'learning machine\' developed by Raytheon Company in the early 1960s?",\n        "options": {\n            "a": "Cybertron",\n            "b": "HAL 9000",\n            "c": "Deep Blue",\n            "d": "R2-D2"\n        },\n        "correct": "a"\n    },\n    "3": {\n        "mcq": "Who proposed the early mathematical models of neural networks for machine learning?",\n        "options": {\n            "a": "Donald Hebb",\n            "b": "Warren McCulloch",\n            "c": "Tom M. Mitchell",\n            "d": "Arthur Samuel"\n        },\n        "correct": "b"\n    },\n    "4": {\n        "mcq": "What is the main objective of modern-day

In [24]:
quiz=json.loads(quiz)

In [25]:
quiz

{'1': {'mcq': 'Who coined the term machine learning in 1959?',
  'options': {'a': 'Donald Hebb',
   'b': 'Tom M. Mitchell',
   'c': 'Arthur Samuel',
   'd': 'Walter Pitts'},
  'correct': 'c'},
 '2': {'mcq': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'options': {'a': 'Cybertron',
   'b': 'HAL 9000',
   'c': 'Deep Blue',
   'd': 'R2-D2'},
  'correct': 'a'},
 '3': {'mcq': 'Who proposed the early mathematical models of neural networks for machine learning?',
  'options': {'a': 'Donald Hebb',
   'b': 'Warren McCulloch',
   'c': 'Tom M. Mitchell',
   'd': 'Arthur Samuel'},
  'correct': 'b'},
 '4': {'mcq': 'What is the main objective of modern-day machine learning?',
  'options': {'a': 'To confuse data',
   'b': 'To make predictions for future outcomes',
   'c': 'To create chaos',
   'd': 'To hide information'},
  'correct': 'b'},
 '5': {'mcq': 'According to Tom M. Mitchell, what is the definition of a computer program learn

In [26]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [27]:
quiz_table_data

[{'MCQ': 'Who coined the term machine learning in 1959?',
  'Choices': 'a: Donald Hebb | b: Tom M. Mitchell | c: Arthur Samuel | d: Walter Pitts',
  'Correct': 'c'},
 {'MCQ': "What was the name of the experimental 'learning machine' developed by Raytheon Company in the early 1960s?",
  'Choices': 'a: Cybertron | b: HAL 9000 | c: Deep Blue | d: R2-D2',
  'Correct': 'a'},
 {'MCQ': 'Who proposed the early mathematical models of neural networks for machine learning?',
  'Choices': 'a: Donald Hebb | b: Warren McCulloch | c: Tom M. Mitchell | d: Arthur Samuel',
  'Correct': 'b'},
 {'MCQ': 'What is the main objective of modern-day machine learning?',
  'Choices': 'a: To confuse data | b: To make predictions for future outcomes | c: To create chaos | d: To hide information',
  'Correct': 'b'},
 {'MCQ': 'According to Tom M. Mitchell, what is the definition of a computer program learning from experience?',
  'Choices': 'a: Its performance at tasks in T, as measured by P, improves with experience

In [28]:
quiz=pd.DataFrame(quiz_table_data)

In [29]:
quiz

,MCQ,Choices,Correct
0,Who coined the term machine learning in 1959?,a: Donald Hebb | b: Tom M. Mitchell | c: Arthu...,c
1,What was the name of the experimental 'learnin...,a: Cybertron | b: HAL 9000 | c: Deep Blue | d:...,a
2,Who proposed the early mathematical models of ...,a: Donald Hebb | b: Warren McCulloch | c: Tom ...,b
3,What is the main objective of modern-day machi...,a: To confuse data | b: To make predictions fo...,b
4,"According to Tom M. Mitchell, what is the defi...","a: Its performance at tasks in T, as measured ...",a


In [30]:
quiz.to_csv("mcq_data",index=False)

In [2]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')
import os

In [4]:
log_path=os.path.join(os.getcwd(),"logs")
os.makedirs(log_path, exist_ok=True)
log_path

'd:\\work\\projects\\project_ChatBot\\experinments\\logs'

In [5]:
os.getcwd()

'd:\\work\\projects\\project_ChatBot\\experinments'

In [6]:
import logging


In [8]:
logging.INFO

20

In [2]:
import os
os.getcwd()

'd:\\work\\projects\\project_ChatBot\\experinments'

In [3]:
os.chdir("d:\\work\\projects\\project_ChatBot")

In [4]:
os.getcwd()

'd:\\work\\projects\\project_ChatBot'